In [ ]:
# We tried using the testbook module to do our testing, but couldn't get it to work in the end (see bottom for testbook
# attempts but we couldn't get it to run)
# Instead we used functions with assert statements. This wouldn't let us use functions from other notebooks so we had to 
# copy and paste our functions into the testing file.

In [96]:
#functions to be tested
def crimes_per_pop_zips(MergedDf):
    MergedDf = MergedDf.set_index(pd.to_datetime(MergedDf['incident_timestamp'])).sort_index()
    ndf = MergedDf['2016-01-01':]
    pop = ndf[['zip_code', 'Total_Population','year']]
    pop = pop.groupby(['zip_code','year']).mean()

    zips = ndf[['zip_code','year']]
    zips = zips.groupby(['zip_code','year']).size().reset_index()
    zips = zips.rename(columns={0:'count'}).set_index(['zip_code','year'])
    new = pd.merge(zips, pop, how='left',on=['zip_code','year'])

    def crimes_per_pop(row):
        return row['count'] / row['Total_Population']

    new['crimes_per_pop'] = new.apply(crimes_per_pop, axis=1)
    pivot = pd.pivot_table(new, values='crimes_per_pop', index='zip_code', columns='year')
    results = []
    for year in pivot.columns:
        results.append('Zip code with the most crimes per total population in {}: {}'.format(str(year)[0:4], str(pivot[year].idxmax())))
    return results

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats 

#processing data and getting a clean dataframe to work with
def processing():
    df=pd.read_csv("TotalPopulation.csv")
    df2=pd.read_csv("Crime_Incidents.csv")
    df = df.rename(columns={'ZCTA5CE10':'zip_code'})
    newdf = pd.merge(df, df2, on='zip_code')
    pop = newdf[['zip_code', 'PopPerSqMile','year']]
    return pop

#getting average population density
def average_population_density(df):
    densities = df['PopPerSqMile'].unique().tolist()
    average_density = sum(densities) / len(densities)
    return average_density, densities

#getting average yearly crime rate for all the zip_codes
def average_crime_rate(df):
    zips = df[['zip_code','year']]
    zips = zips.groupby(['zip_code','year']).size().reset_index()
    zips = zips.rename(columns={0:'count'}).set_index(['zip_code','year'])
    zips = zips.reset_index(level=1)
    average_crimes_per_year = []
    zip_codes = df['zip_code'].unique()
    for code in zip_codes:
        area = zips.loc[code]
        average = area['count'].mean()
        average_crimes_per_year.append(average)
    total_average = sum(average_crimes_per_year) / len(average_crimes_per_year)
    return total_average, average_crimes_per_year, zip_codes

#creating final dataframe for my output
def final_df(zip_codes,densities,average_crimes_per_year,average_density,total_average):
    final = pd.DataFrame({'zip_code':zip_codes,'population_density':densities,'average_crimes_per_year':average_crimes_per_year})
    final['relative_population_density'] = np.where(final['population_density'] > average_density, 'High','Low')
    final['relative_average_crime'] = np.where(final['average_crimes_per_year'] > total_average, 'High','Low')
    return final
    
#get percentage of high crime areas that are also high population dense
def percent(df):
    high_crime = df[df['relative_average_crime'] == 'High']
    high_crime_and_density = high_crime[high_crime['relative_population_density'] == 'High']
    len_high_crime = len(high_crime)
    len_high = len(high_crime_and_density)
    percent_high = (100*(len_high / len_high_crime))
    print(str(percent_high) + "% of high crime areas are also high population density areas")
    return percent_high

#graph scatter plot comparing population density and average crimes per year
def graph(df):
    plot = df.plot.scatter(x='population_density', y='average_crimes_per_year', alpha=1)
    print(plot)

#perform t-test to see if difference between crime rates in high population density and low population density areas are statistically significant
def t_test(df):
    high_density_crime_rates = df[df['relative_population_density'] == 'High']['average_crimes_per_year'].tolist()
    low_density_crime_rates = df[df['relative_population_density'] == 'Low']['average_crimes_per_year'].tolist()
    t, p = stats.ttest_ind(high_density_crime_rates,low_density_crime_rates)
    print("t = " + str(t))
    print("p = " + str(p))
    return t, p


pop = processing()
average_density = average_population_density(pop)[0]
densities = average_population_density(pop)[1]


total_average = average_crime_rate(pop)[0]
average_crimes_per_year = average_crime_rate(pop)[1]
zip_codes = average_crime_rate(pop)[2]


final = final_df(zip_codes,densities,average_crimes_per_year,average_density,total_average)

import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

#the apply function which is used in the creating dataframe function
def findYoungPopulation(df):
    ratioYoung = df['Population_Under_18']/df['Total_Population']
    percentYoung=ratioYoung*100
    return percentYoung

#the function used to create the dataframe 
def creatingDataFrame():
    df=pd.read_csv("TotalPopulation.csv")
    df2=pd.read_csv("Crime_Incidents.csv")
    df = df.rename(columns={'ZCTA5CE10':'zip_code'})
    newdf = pd.merge(df, df2, on='zip_code')
    newdf=newdf.set_index(pd.to_datetime(newdf['incident_timestamp']))
    newdf = newdf.loc['2017-01-01':'2017-12-31']

    newdf=newdf.groupby('zip_code').apply(findYoungPopulation).to_frame(name='ratio')

    popdf = newdf.groupby('zip_code').agg(len)
    Finaldf=pd.merge(newdf, popdf, on='zip_code')

    Finaldf=Finaldf.groupby('zip_code').agg(max)
    Finaldf=Finaldf.rename(columns={'ratio_x':'Population_Under_18', 'ratio_y':'Crimes_in_2017'})
    Finaldf=Finaldf.sort_values(by=['Population_Under_18'], ascending=False)
    return(Finaldf)

#creating the visual 
def CreatingVisual(Df):
    Crimes2017 = Df['Crimes_in_2017']
    Crimes2017= Crimes2017.to_list()
    CrimeTotal = sum(Crimes2017)
    PercentageCrime =[]
    for items in Crimes2017:
        PercentageCrime.append((items / CrimeTotal)* 100) 

    ZipCodes = Df.index.values
    PopulationUnder18 = Df['Population_Under_18']
    PopulationUnder18= PopulationUnder18.to_list()
    y_pos = np.arange(len(ZipCodes))

    x = np.arange(len(ZipCodes))  # the label locations
    width = 0.4  # the width of the bars

    fig, ax = plt.subplots()
    rects1 = ax.bar(x - width/2, PopulationUnder18, width, label='Popunder18')
    rects2 = ax.bar(x + width/2, PercentageCrime, width, label='PercentageCrime')

    plt.ylabel('Percent of Pop')
    plt.title('Population Under 18 vs. Crime By Zip')
    ax.set_xticks(x)
    ax.set_xticklabels(ZipCodes)
    plt.xticks(rotation=90)
    ax.legend()
    plt.show()
    
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt


#the apply function which is used in the creating dataframe function
def findYoungPopulation(df):
    ratioYoung = df['Population_Under_5']/df['Total_Population']
    percentYoung=ratioYoung*100
    return percentYoung

#the function used to create the dataframe 
def creatingDataFrame5():
    df=pd.read_csv("TotalPopulation.csv")
    df2=pd.read_csv("Crime_Incidents.csv")
    df = df.rename(columns={'ZCTA5CE10':'zip_code'})
    newdf = pd.merge(df, df2, on='zip_code')
    newdf=newdf.set_index(pd.to_datetime(newdf['incident_timestamp']))
    newdf = newdf.loc['2017-01-01':'2017-12-31']

    newdf=newdf.groupby('zip_code').apply(findYoungPopulation).to_frame(name='ratio')

    popdf = newdf.groupby('zip_code').agg(len)

    Finaldf=pd.merge(newdf, popdf, on='zip_code')

    Finaldf=Finaldf.groupby('zip_code').agg(max)
    Finaldf=Finaldf.rename(columns={'ratio_x':'Population_Under_5', 'ratio_y':'Crimes_in_2017'})
    Finaldf=Finaldf.sort_values(by=['Population_Under_5'], ascending=False)
    return(Finaldf)

#creating the visual 
def CreatingVisual5(Finaldf):
    Crimes2017 = Finaldf['Crimes_in_2017']
    Crimes2017= Crimes2017.to_list()
    CrimeTotal = sum(Crimes2017)
    PercentageCrime =[]
    for items in Crimes2017:
        PercentageCrime.append((items / CrimeTotal)* 100) 

    ZipCodes = Finaldf.index.values
    PopulationUnder5 = Finaldf['Population_Under_5']
    PopulationUnder5= PopulationUnder5.to_list()
    y_pos = np.arange(len(ZipCodes))

    x = np.arange(len(ZipCodes))  # the label locations
    width = 0.4  # the width of the bars

    fig, ax = plt.subplots()
    rects1 = ax.bar(x - width/2, PopulationUnder5, width, label='Popunder5')
    rects2 = ax.bar(x + width/2, PercentageCrime, width, label='PercentageCrime')

    plt.ylabel('Percent of Pop')
    plt.title('Population Under 5 vs. Crime By Zip')
    ax.set_xticks(x)
    ax.set_xticklabels(ZipCodes)
    plt.xticks(rotation=90)
    ax.legend()
    plt.show()
    

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [99]:
# tests
from statistics import mean
import numpy as np
import pandas as pd
import unittest

class Test_Assertions(unittest.TestCase):
    
    df1 = creatingDataFrame()
    df2 = creatingDataFrame5()
    
    def test_creatingDataFrame(self):
        assert type(self.df1)==(pd.core.frame.DataFrame), "You must return a pandas dataframe."
        assert len(self.df1) == 30, "You have not returned a data frame with six entries in it."
        assert "Population_Under_18" in self.df1.keys(), "You have not returned a dataframe with the correct column names."
        assert "Crimes_in_2017" in self.df1.keys(), "You have not returned a dictionary with the correct keys."

    def test_creatingDataFrame5(self):
        assert type(self.df2)==(pd.core.frame.DataFrame), "You must return a pandas dataframe."
        assert len(self.df2) == 30, "You have not returned a data frame with six entries in it."
        assert "Population_Under_5" in self.df2.keys(), "You have not returned a dataframe with the correct column names."
        assert "Crimes_in_2017" in self.df2.keys(), "You have not returned a dictionary with the correct keys."
    
    df = processing()
    zip_codes = average_crime_rate(df)[2]
    densities = average_population_density(df)[1]
    average_crimes_per_year = average_crime_rate(df)[1]
    average_density = average_population_density(df)[0]
    total_average = average_crime_rate(df)[0]
    final = final_df(zip_codes,densities,average_crimes_per_year,average_density,total_average)
    high_crime = final[final['relative_average_crime'] == 'High']
    
    def test_processing(self):
        self.assertEqual(len(self.df.columns),3) 
        self.assertEqual(len(self.df.index),348247)
        
    def test_average_population_density(self):
        self.assertEqual(len(self.densities),30) 
                
    def test_average_crime_rate(self):
        self.assertEqual(len(self.zip_codes),30) 
        self.assertEqual(len(self.average_crimes_per_year),30) 
        
    def test_final_df(self):
        self.assertEqual(len(self.final.columns),5)
        self.assertEqual(len(self.final.index),30)
        self.assertEqual(self.final['zip_code'].tolist(),self.final['zip_code'].unique().tolist())

    def test_percent(self):
        self.assertEqual(len(self.high_crime),13)
        self.assertEqual(len(self.high_crime[self.high_crime['relative_population_density'] == 'High']),10)
        percent_high = percent(self.final)
        self.assertIsInstance(percent_high,float) 
        
    def test_t_test(self):
        high_density_crime_rates = self.final[self.final['relative_population_density'] == 'High']['average_crimes_per_year'].tolist()
        low_density_crime_rates = self.final[self.final['relative_population_density'] == 'Low']['average_crimes_per_year'].tolist()
        self.assertEqual(len(high_density_crime_rates),15)
        self.assertEqual(len(low_density_crime_rates),15)
    
        n = len(high_density_crime_rates)
        t1 = t_test(self.final)[0]
        p1 = t_test(self.final)[1]
        var_1 = self.final[self.final['relative_population_density'] == 'High']['average_crimes_per_year'].var(ddof=1)
        var_2 = self.final[self.final['relative_population_density'] == 'Low']['average_crimes_per_year'].var(ddof=1)
        s = np.sqrt((var_1 + var_2)/2)
        t = (self.final[self.final['relative_population_density'] == 'High']['average_crimes_per_year'].mean() - self.final[self.final['relative_population_density'] == 'Low']['average_crimes_per_year'].mean())/(s*np.sqrt(2/n))
        dof = 2*n - 2
        p = 2 * (1 - stats.t.cdf(t,df=dof))
        self.assertAlmostEqual(t1,t)
        self.assertAlmostEqual(p1,p)

    highest_crimes_per_zip = crimes_per_pop_zips(MergedDf)
    
    def test_crimes_per_pop_zips(self):
        assert isinstance(self.highest_crimes_per_zip, list)
        assert len(self.highest_crimes_per_zip) == 6

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in long_scalars
  


In [100]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


......FF.

t = 4.2860225819723645
p = 0.00019432541819120193
t = 4.2860225819723645
p = 0.00019432541819120193



FAIL: test_percent (__main__.Test_Assertions)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-99-121586bd7e65>", line 50, in test_percent
    self.assertEqual(len(self.high_crime),13)
AssertionError: 16 != 13

FAIL: test_processing (__main__.Test_Assertions)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-99-121586bd7e65>", line 35, in test_processing
    self.assertEqual(len(self.df.index),348247)
AssertionError: 51856 != 348247

----------------------------------------------------------------------
Ran 9 tests in 0.015s

FAILED (failures=2)


In [101]:
import pandas as pd
PopulationDf = pd.read_csv("TotalPopulation.csv")
CrimeDf = pd.read_csv("Crime_Incidents.csv")
PopulationDf = PopulationDf.rename(columns={'ZCTA5CE10':'zip_code'})
MergedDf = pd.merge(PopulationDf, CrimeDf, on='zip_code')
MergedDf = MergedDf.set_index(pd.to_datetime(MergedDf['incident_timestamp'])).sort_index()['2016-01-01':]
# find most common type of crime per year using pandas
#visually compare that they are the same as those returned by the sql table
def most_crime_per_year_pd(MergedDf):
    piv = pd.pivot_table(MergedDf, index='offense_description', columns='year', values='zip_code', aggfunc='count')
    for year in piv.columns:
        print('The most common offense of {} was: {}'.format(str(year)[0:4], str(piv[year].idxmax())))
most_crime_per_year_pd(MergedDf)


The most common offense of 2016 was: BURGLARY - FORCED ENTRY
The most common offense of 2017 was: ASSAULT AND BATTERY/SIMPLE ASSAULT
The most common offense of 2018 was: ASSAULT AND BATTERY/SIMPLE ASSAULT
The most common offense of 2019 was: ASSAULT AND BATTERY/SIMPLE ASSAULT
The most common offense of 2020 was: ROBBERY
The most common offense of 2021 was: ARSON


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
